# Get Programming with F# by [Isaac Abraham](https://github.com/isaacabraham)

## “CAPSTONE 3”

This capstone is an opportunity to use what we have learned in the previous lessons (specifically the last four) to get rid of the `mutable` shenanigans in [Capstone 2](https://github.com/BryanWilhite/jupyter-central/blob/master/get-programming-with-f-sharp/14-capstone-02.ipynb).

>There are two ways of avoiding the imperative mutable model:
>
>- using recursion, which I’ve deliberately avoided so far.
>- Treating the changes to the account as a sequence of operations that are applied against the previous version of the account…
>

My notes for Capstone 2 addresses the need for “a sequence of operations” with:

```fsharp
account
|> withdraw 50M
|> deposit 50M 
|> deposit 100M 
|> withdraw 50M 
|> withdraw 350M
```

But Capstone 3 demands support for a ‘stream’ of commands, say:

In [ ]:
#!fsharp

let commands: (char * Option<decimal>) list =
    [ 
        ('w', Some 50M) // I am simplifying what we have in the text by using command-amount tuples
        ('d', Some 50M)
        ('d', Some 100M)
        ('w', Some 50M)
        ('j', Some -9M) // this implies the need to validate commands
        ('w', Some 350M)
        ('x', None) //exit (terminate journal)
    ]


The use of `Some` [📖 [docs](https://fsharp.github.io/fsharp-core-docs/reference/fsharp-core-fsharpoption-1.html#Some)] in the tuples above allow an amount to be `None` which is handy for the `x` command.

I will reuse the code from Capstone 2 for the sake of my notes:

In [ ]:
#!fsharp

#load "14-capstone-02/Domain.fs"
#load "14-capstone-02/Operations.fs"
#load "19-capstone-03/Operations.fs"
#load "14-capstone-02/Auditing.fs"

open Capstone2.Operations
open Capstone2.Domain
open Capstone2.Auditing
open System

let withdraw = auditAs "withdraw" console withdraw
let deposit = auditAs "deposit" console deposit

let processCommand account (command, amount) =
    printfn ""
    let account =
        if command = 'd' then account |> deposit amount
        else account |> withdraw amount
    printfn "Current balance is £%M" account.Balance
    account

let customer = { Name = "Isaac" }
let openingAccount = { AccountId = Guid.Empty; Owner = customer; Balance = 0M }

Along with the code reuse, a new `Operations.fs` file is loaded:

In [ ]:
Get-Content -Path ./19-capstone-03/Operations.fs

[<AutoOpen>]
module CommandParsing

    let isValidCommand (cmd, amt) = [ 'd'; 'w'; 'x' ] |> List.contains cmd
    let isValidAmount (cmd, amt) = Some(amt).IsSome
    let isValidCommandAndAmount (cmd, amt) =
        [
            isValidCommand
            isValidAmount
        ]
        |> List.map (fun f -> f (cmd, amt)) 
        |> List.reduce (fun previous current -> previous && current)

    let isStopCommand (cmd, amt) = cmd = 'x'


In [ ]:
#!fsharp

let closingAccount =
    commands
    |> List.filter isValidCommandAndAmount
    |> List.takeWhile (not << isStopCommand) // `<<` is the backward composition operator
    |> List.map (fun (cmd, amt) -> (cmd, amt.Value))
    |> List.fold processCommand openingAccount


Account 00000000-0000-0000-0000-000000000000: 10/24/2021 6:29:05 AM: Performing a withdraw operation for £50...
Account 00000000-0000-0000-0000-000000000000: 10/24/2021 6:29:05 AM: Transaction rejected!
Current balance is £0

Account 00000000-0000-0000-0000-000000000000: 10/24/2021 6:29:05 AM: Performing a deposit operation for £50...
Account 00000000-0000-0000-0000-000000000000: 10/24/2021 6:29:05 AM: Transaction accepted! Balance is now £50.
Current balance is £50

Account 00000000-0000-0000-0000-000000000000: 10/24/2021 6:29:05 AM: Performing a deposit operation for £100...
Account 00000000-0000-0000-0000-000000000000: 10/24/2021 6:29:05 AM: Transaction accepted! Balance is now £150.
Current balance is £150

Account 00000000-0000-0000-0000-000000000000: 10/24/2021 6:29:05 AM: Performing a withdraw operation for £50...
Account 00000000-0000-0000-0000-000000000000: 10/24/2021 6:29:05 AM: Transaction accepted! Balance is now £100.
Current balance is £100

Account 00000000-0000-0000-00

@[BryanWilhite](https://twitter.com/BryanWilhite)


In [ ]:
#!about

.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.250604+e99bd250539a64e828b44cb9ad9353eeda9b5bc1Build date: 2021-10-07T01:11:06.0000000Zhttps://github.com/dotnet/interactive
